In [ ]:
import pandas as pd
import numpy as np
from xml.etree import ElementTree as etree
import os, sys, re 


# file_path = r'L:\sbharadwaj\TGL81_TPI_VCC_Continuity'
# csv_file = r'TGL81_16Weeks_Data.csv'
# mtpl_file = r'TPI_VCC_31N.mtpl'
# xml_file = r'VCC_Parallel_TGL81.xml'

file_path = r'L:\sbharadwaj\TGL42_TPI_VCC_Continuity'
csv_file = r'TGL42_Data_1week.csv'
mtpl_file = r'TPI_VCC_13N.mtpl'
xml_file = r'TGL42_B0_VCC_Parallel.xml'

data0 = pd.read_csv(file_path +'\\'+ csv_file)
data0 = data0[data0.columns.drop(list(data0.filter(regex = 'PRESURGE')))] #Retains only VCC sources. Remove if you need PreSurge data
#data0 = data0[data0.columns.drop(list(data0.filter(regex = 'CONT_HIGH')))] #Retains only VCC sources. Remove if you need High Res sources
fMTPL = (file_path +'\\'+ mtpl_file)
#data0 = data0.head(1000) 

passbin = [1,2,3,4]

for col in data0.columns:
    if 'IB@D1D' in col :
      
        #data1 = data0[data0[col].str.contains('|'.join(passbin), regex = True)]
        #data1 = data0[data0[col].str.contains('1|2|3|4', regex = True)]
        #data1 = data0[data0[col].astype(int).contains('1|2|3|4', regex = True)]
        #print(data0[data0[col]])
        data1 = data0[data0[col].isin(passbin)]

try:
    data1.to_csv(file_path +'\\'+'Debug.csv')
except:
    print('Close Debug.csv file')
    
    

In [91]:
stat = data1.describe(include = 'all')

cParamList = []
cTestBegin1 = []
cTestBegin = []

for col in stat.columns:
    cParamList.append(col)

for cTestBeginGen in cParamList:
    cTestBeginGen = str.split(cTestBeginGen,'::') 
    cTestBeginGen = list(filter(lambda x: len(cTestBeginGen)>1, cTestBeginGen))
    if (len(cTestBeginGen)) > 0 :
        cTestBegin1.append(cTestBeginGen)

cTestBegin1 = [x[1] for x in cTestBegin1]
cTestBeginGen = []

for cTestBeginGen in cTestBegin1:
    cTestBeginGen = str.split(cTestBeginGen,' ')
    cTestBegin.append(cTestBeginGen)

cTestBegin = [x[0] for x in cTestBegin]    

#print((cTestBegin[-1]))



In [92]:

try:
    stat.to_csv(file_path +'\\'+'stat.csv')
except:
    print('Close stat.csv file')

In [93]:
pTestBegin = re.compile(r'^Test (?P<template>\w+) (?P<testName>[A-Za-z0-9_]+)')
plimit_low = re.compile(r'limit_low = "(?P<limit_low_result>.+)";')
plimit_high = re.compile(r'limit_high = "(?P<limit_high_result>.+)";')
pEndTest = re.compile(r'^}')

pullData = False
limit_high = "NoValue"
limit_low = "NoValue"
testList = []
#testList = r"Parametric,High Value,Low Value"
#testList = [['Parametric'],['High Value'],['Low Value']]

#testList   = testList.split(',')

with open(os.path.join(file_path,mtpl_file),'r') as lines:
    for line in lines:
        
        matchBegin = re.search(pTestBegin,line)
        if matchBegin:
            currTest = matchBegin.group("testName")
            pullData = True     

        match_high_limit = re.search(plimit_high,line)
        if match_high_limit and pullData:
            limit_high = match_high_limit.group("limit_high_result")#.replace(","," ")

        match_low_limit = re.search(plimit_low,line)
        if match_low_limit and pullData:
            limit_low = match_low_limit.group("limit_low_result")#.replace(","," ")
            
        matchEND = re.search(pEndTest,line)
        if matchEND and pullData:
            testItem = "{},{},{}".format(currTest,limit_high,limit_low)
            testItem   = testItem.split(',')
            testList.append(testItem)
            
            pullData = False
            limit_high = "NoValue"
            limit_low = "NoValue"


highLow = pd.DataFrame(testList)
highLow.rename(columns = {0:'Parametric', 1:'High Value', 2:'Low Value'}, inplace = True)
highLow = highLow[highLow['High Value'] != 'NoValue']

highLow = highLow.set_index('Parametric').T
highLow = highLow.reindex(sorted(highLow.columns), axis = 1)


# try:
#     highLow.to_csv(file_path +'\\'+'hiLow.csv')
# except:
#     print('Close highLow.csv file')

In [94]:
xml_Limits = {}
vcc_xml = etree.parse(file_path +'\\'+'VCC_Parallel_TGL81.xml')
root = vcc_xml.getroot()
ConfigList = root.findall('ConfigList')

for child in ConfigList:
    if 'START' in child.attrib['name']:
        #for node in child.getiterator():
        for node in child.iter():
            if node.tag == 'Pin':
                xml_Limits.setdefault('Pin',[]).append(node.text)
            if node.tag == 'limit_high':
                xml_Limits.setdefault('High Value',[]).append(node.text)
            if node.tag == 'limit_low':
                xml_Limits.setdefault('Low Value',[]).append(node.text)
            #print(node.tag)
            #print(node.text)

xml_highLow = pd.DataFrame.from_dict(xml_Limits)
xml_highLow = xml_highLow.T
xml_highLow.columns = xml_highLow.loc['Pin']
xml_highLow = xml_highLow.reindex(sorted(xml_highLow.columns), axis = 1)
print(xml_highLow)

Pin         VCC_1P8_LC  VCC_ANAEHV_LC  VCC_C01_HC  VCC_C23_HC  VCC_C45_HC  \
Pin         VCC_1P8_LC  VCC_ANAEHV_LC  VCC_C01_HC  VCC_C23_HC  VCC_C45_HC   
High Value     0.00069        0.00065       0.024       0.022       0.023   
Low Value       -0.001         -0.001       -0.01       -0.01       -0.01   

Pin         VCC_C67_HC  VCC_DD2_LC  VCC_DDQ_LC  VCC_GT_HC  VCC_INAUX_HC  \
Pin         VCC_C67_HC  VCC_DD2_LC  VCC_DDQ_LC  VCC_GT_HC  VCC_INAUX_HC   
High Value       0.024     0.00027      0.0028      0.024        0.0065   
Low Value        -0.01      -0.001       -0.01      -0.01         -0.01   

Pin         VCC_IN_HC  VCC_IO_HC  VCC_PCIE_LC  VCC_RING_HC  VCC_SA_HC  \
Pin         VCC_IN_HC  VCC_IO_HC  VCC_PCIE_LC  VCC_RING_HC  VCC_SA_HC   
High Value     0.0125      0.024        0.008        0.024      0.024   
Low Value       -0.01      -0.01       -0.001        -0.01      -0.01   

Pin         VCC_STGFUSE_LC  VCC_STG_LC  VCC_ST_LC  VCC_TCPHY_LC  
Pin         VCC_STGFUSE_LC  VCC

In [95]:
limits = stat

for highLow_col in highLow.columns:
    for stat_col in stat.columns:
        if highLow_col in stat_col:

            high_value = highLow.at['High Value', highLow_col]
            low_value = highLow.at['Low Value', highLow_col]
            limits.at['High Value', stat_col ] = high_value
            limits.at['Low Value', stat_col] = low_value

for xml_highLow_col in xml_highLow.columns:
    for limits_col in limits.columns:
        if xml_highLow_col in limits_col and 'HIGHRES' in limits_col:
            
            
            high_value = xml_highLow.at['High Value', xml_highLow_col]
            low_value = xml_highLow.at['Low Value', xml_highLow_col]
            limits.at['High Value', limits_col ] = high_value
            limits.at['Low Value', limits_col] = low_value
            
limits  = limits.T

#limits['Mean + 6 sigma'] = limits['Mean'].value + 6* limits['std'].value
#             limits = pd.concat(stat, highLow, left_on= stat_col, right_on= highLow_col )
#             pd.limits = stat.merge(highLow.rename(columns={highLow_col:stat_col} ), how = 'outer')
#             limits = pd.concat([stat, highLow.rename(columns = {highLow_col:stat_col})], ignore_index = False, sort = True)

try:
    limits.to_csv(file_path +'\\'+'limits.csv')
except:
    print('Close limits.csv file')
    

